# Big Data Platform
## Assignment 2: MapReduce

**By:**  

Or Livne, 300123123  
Roy Rubin, 201312907

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of MapReduceEngine

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a Jupyter notebook.
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.
  
  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q1 - 5 points - Initial Steps
- Q2 - 50 points - MapReduceEngine
- Q3 - 30 points - Implement the MapReduce Inverted index of the JSON documents
- Q4 - 5 points - Testing Your MapReduce
- Q5 - 10 points - Final Thoughts 

`Total: 100`

**Prerequisites**

In [3]:
import sys
print(f'\nInstall joblib\n')
!{sys.executable} -m pip install joblib
#!pip install joblib
print(f'\nInstall lithops\n')
!{sys.executable} -m pip install lithops


Install joblib


Install lithops



**Imports**

In [4]:
#SQL
import sqlite3

# ML
import pandas as pd
import numpy as np 

# general
import os
import time
import random
import warnings
import string 
import glob
from shutil import copyfile
import shutil
from pathlib import Path
from functools import partial

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display

# threading
from joblib import Parallel, delayed

# lithops
import lithops

In [25]:
!python --version

Python 3.8.5


**Hide Warnings**

In [5]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [6]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Set Random Seed**

In [7]:
random.seed(123)

**Assisting Functions**

In [8]:
def init_configurations():
    # Set the value options
    global max_rows, csv_columns, db_file_name, csv_file_name, columns_type
    global csv_index, csv_ending, amount_of_files
    global map_reduce_folder_names, amount_of_process, map_regex, db_columns
    global db_columns_type, db_table_name, reduce_regex_init, reduce_regex_final
    
    
    Current_python_file_path = os.getcwd()
    max_rows = 10
    amount_of_files = 20
    amount_of_process = 2
    csv_columns = ['firstname','secondname','city']
    db_columns = ['key', 'value']
    map_reduce_folder_names =  [os.path.join(Current_python_file_path, 'mapreducetemp'), 
                                os.path.join(Current_python_file_path, 'mapreducefinal')]

    db_columns_type = [ 'text',  'text']

    db_file_name = 'mydata.db'
    csv_file_name = 'myCSV'
    csv_ending = '.csv'
    map_regex = 'part-tmp-'
    reduce_regex_init = 'part-'
    reduce_regex_final = '-final'

    db_table_name = 'temp_results'

    return


In [9]:
# set relevant configurations
init_configurations()

<br><br><br><br>
# Question 1
# Initial Steps

Write Python code to create 20 different CSV files in this format:  `myCSV[Number].csv`, where each file contains 10 records. 

The schema is `(‘firstname’,’secondname’,city’)`  

Values should be randomly chosen from the lists: 
- `firstname` : `[John, Dana, Scott, Marc, Steven, Michael, Albert, Johanna]`  
- `city` : `[New York, Haifa, München, London, Palo Alto,  Tel Aviv, Kiel, Hamburg]`  
- `secondname`: any value  

In [10]:
class single_record():
    firstname = ''
    secoundname = ''
    city = ''
    
    # Set the value options
    firstname_option_list  = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
    secoundname_option_list  = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
    city_option_list = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto',  'TelAviv', 'Kiel', 'Hamburg']
    def __init__(self, id):
        self.firstname = random.choice(self.firstname_option_list)	
        random_name  = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase) for _ in range(random.randint(2,4)))
        self.secoundname = random_name
        self.city = random.choice(self.city_option_list)

def create_csvdatabase_file(max_rows):
    # this loop generate single fruit
    create_n_list_in_advance = max_rows*[None]
    for i_row_index in range(max_rows):
        i_record = single_record(i_row_index)
        create_n_list_in_advance[i_row_index] = [i_record.firstname, i_record.secoundname, i_record.city]
    
    mydata_df = pd.DataFrame(create_n_list_in_advance, columns = csv_columns)
    #mydata_df.to_csv(csv_file_name)  #TODO verify ROY commented this
    return mydata_df

def generate_n_csv_files():
    for i_csv_index in range(0, amount_of_files):
        mydata_df = create_csvdatabase_file(max_rows)
        mydata_df.to_csv(csv_file_name + str(i_csv_index) + csv_ending)
    return

In [11]:
# generate all csv files
generate_n_csv_files()

Use python to Create `mapreducetemp` and `mapreducefinal` folders

In [12]:
def create_new_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    return 

def generate_map_reduce_folders():
    for i_folder in map_reduce_folder_names:
        print(f'Creating folder: {i_folder}')
        create_new_folder(i_folder)
    return 

In [13]:
# generate folders
generate_map_reduce_folders()

Creating folder: /Users/royrubin/PycharmProjects/BigDataPlatforms/HW3/mapreducetemp
Creating folder: /Users/royrubin/PycharmProjects/BigDataPlatforms/HW3/mapreducefinal


<br><br><br>
# Question 2
## MapReduceEngine

1. Write Python code to create an SQLite database with the following table

`TableName: temp_results`   
`schema: (key:TEXT,value:TEXT)`

In [14]:
def create_database(db_file_name):
    is_exist = os.path.exists(db_file_name)
    if  is_exist:
        os.remove(db_file_name)
    con = sqlite3.connect(db_file_name)
        
    
    cur = con.cursor()
    
    # Create table
    columns_type_list = list(map(lambda x,y: x+' ' + y, db_columns, db_columns_type))
    columns_type_list_string = "("+", ".join(map(str, columns_type_list))+")"

    cur.execute(''' 
                CREATE TABLE temp_results
                ''' + columns_type_list_string + \
               '''''')
    
    con.commit()
    con.close()
    return con, cur

In [15]:
sql_conn, cur = create_database(db_file_name)

2. **Create a Python class** `MapReduceEngine` with method `def execute(input_data, map_function, reduce_function)`, such that:
    - `input_data`: is an array of elements
    - `map_function`: is a pointer to the Python function that returns a list where each entry of the form (key,value) 
    - `reduce_function`: is pointer to the Python function that returns a list where each entry of the form (key,value)

<br><br>

3.**Implement** the following functionality in the `execute(...)` function:

<br>

1. For each key  from the  input_data, start a new Python thread that executes map_function(key) 
<br><br>
2. Each thread will store results of the map_function into mapreducetemp/part-tmp-X.csv where X is a unique number per each thread.
<br><br>
3. Keep the list of all threads and check whether they are completed.
<br><br>
4. Once all threads completed, load content of all CSV files into the temp_results table in SQLite.

    Remark: Easiest way to loop over all CSV files and load them into Pandas first, then load into SQLite  
    `data = pd.read_csv(path to csv)`  
    `data.to_sql(‘temp_results’,sql_conn, if_exists=’append’,index=False)`
<br><br>

5. **Write SQL statement** that generates a sorted list by key of the form `(key, value)` where value is concatenation of ALL values in the value column that match specific key. For example, if table has records
<table>
    <tbody>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV1.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">Dana</td>
                <td style="text-align:center">myCSV5.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV7.csv</td>
            </tr>
    </tbody>
</table>

    Then SQL statement will return `(‘John’,’myCSV1.csv, myCSV7.csv’)`  
    Remark: use GROUP_CONCAT and also GROUP BY ORDER BY
<br><br><br>
6. **Start a new thread** for each value from the generated list in the previous step, to execute `reduce_function(key,value)` 
<br>    
7. Each thread will store results of reduce_function into `mapreducefinal/part-X-final.csv` file  
<br>
8. Keep list of all threads and check whether they are completed  
<br>
9. Once all threads completed, print on the screen `MapReduce Completed` otherwise print `MapReduce Failed` 



In [21]:
config = ...

In [22]:
# implement all of the class here

class MapReduceEngine():
    def execute(self, input_data, map_function, reduce_function):
        #  1) For each key  from the  input_data, start a new Python thread that executes 
        #     map_function(key)
        #  2) Each thread will store results of the map_function into 
        #     mapreducetemp/part-tmp-X.csv where X
        #     is a unique number per each thread.
        #  3) Keep the list of all threads and check whether they are completed
        
        # succeed_new_path_list  = Parallel(n_jobs=amount_of_process, backend="threading", \
        #                                   prefer="processes")(delayed(self._run_at_parallel_map)(
        #     index, item, map_function) for index, item in enumerate(input_data))
        
        
        fexec = lithops.FunctionExecutor(config=config)
        
        
        for index, item in enumerate(input_data):
            fexec.call_async(self._run_at_parallel_map, index, item, map_function)
            
        
        
        # 4) Once all threads completed, load content of all CSV files into the temp_results
        #    table in SQLite
        
#         # get new files names
#         filepaths = [path for boolean, path in succeed_new_path_list]
        
#         # write generated csv files to sql data base
#         sql_conn = sqlite3.connect(db_file_name)
#         list(map(lambda x: pd.read_csv(x, index_col=0) \
#                  .to_sql('temp_results',sql_conn, if_exists='append',index=False), filepaths))    
#         sql_conn.close()
        
#         # get list of succeed or failed of threads
#         boolean_results = [boolean for boolean, path in succeed_new_path_list]
        
#         # validate that all threads are completed succesfully
#         if False in boolean_results:
#             status = 'Map Reduce Failed'
#             return status

#         #print_db_file_info()
        
#         # 5) **Write SQL statement** that generates a sorted list by key of the form 
#         #    `(key, value)` where value is concatenation of ALL values in the value column
#         #     that match specific key. For example, if table has records
        
#         # query data base using GROUP_CONCAT and GROUP BY  and ORDER BY 
#         generated_list = self._get_grouped_info_from_db_by_key(key='key')
        
#         # 6) **Start a new thread** for each value from
#         #    the generated list in the previous step, to execute `reduce_function(key,value)
#         #    Begin by Performing REDUCE actions
#         #    we will open a thread for each REDUCE
#         # 7) Each thread will store results of reduce_function into 
#         #   `mapreducefinal/part-X-final.csv` file
        
#         # 8) Keep list of all threads and check whether they are completed
        
#         reduce_return_dict_succeed  = Parallel(n_jobs=amount_of_process, backend="threading",\
#                                                prefer="processes")(delayed(self._run_at_parallel_reduce)(
#             index, item, reduce_function) for index, item in enumerate(generated_list))

        
#         # 9) Once all threads completed, print on the screen 
#         #   `MapReduce Completed` otherwise print `MapReduce Failed`
#         boolean_results = [boolean for boolean, path in reduce_return_dict_succeed]
        
#         if False in boolean_results:
#             status = 'Map Reduce Failed'
#             return status
        
#         reduce_dict = [reduce_dict for boolean, reduce_dict in reduce_return_dict_succeed]
    
#         reduce_df = pd.DataFrame(data= reduce_dict)
#         status = 'Map Reduce Completed'
        
        return status

    def _run_at_parallel_map(self, i_item_index, i_item, function):
        new_path = None
        succeed = True
        try:

            new_path  = os.path.join(map_reduce_folder_names[0],
                                    map_regex + str(i_item_index) + csv_ending) 

            dict_result = function(i_item)
            result_df = pd.DataFrame(data=dict_result, columns = ['key', 'value'])

            result_df.to_csv(new_path) 

            succeed = os.path.exists(new_path)
        except:
            succeed = False

        return  succeed, new_path


    def _run_at_parallel_reduce(self, i_item_index, i_item, function):
        new_path = None
        succeed = True
        try:
            new_path  = os.path.join(map_reduce_folder_names[1],
                                     reduce_regex_init + str(i_item_index) + reduce_regex_final + csv_ending)
            #print(new_path)
            key = i_item[0]
            value = i_item[1]

            dict_result = function(key, value)
            result_df = pd.DataFrame.from_records([dict_result])

            result_df.to_csv(new_path) 

            succeed = os.path.exists(new_path)
        except:
            succeed = False

        return  succeed, new_path


    def _get_grouped_info_from_db_by_key(self, key):
        con = sqlite3.connect(db_file_name)
        cur = con.cursor()
        return_list = []
        for row in cur.execute('SELECT key, GROUP_CONCAT(value) FROM ' +db_table_name+ ' GROUP BY ' + key + ' ORDER BY ' + key):
            #print(row)
            return_list.append(row)
        con.close()
        return return_list
    

<br><br><br><br>

# Question 3
## Implement the MapReduce Inverted index of the JSON documents

Implement a function `inverted_map(document_name)` which reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name).

For example, if myCSV4.csv document has values like:  
`{‘firstname’:’John’,‘secondname’:’Rambo’,‘city’:’Palo Alto’}`

Then `inverted_map(‘myCSV4.csv’)` function will return a list:  
`[(‘firstname_John’,’ myCSV4.csv’),(‘secondname_Rambo’,’ myCSV4.csv’), (‘city_Palo Alto’,’ myCSV4.csv’)]`

In [17]:
def inverted_map(document_name):
    #ducument_name = input_data[0]
    csv_df = pd.read_csv(document_name, index_col=0)
    csv_size = csv_df.shape[0]
    csv_columns = csv_df.columns.to_list()
    output_list = []
    for i_col in csv_columns:
        col_vals  = csv_df[i_col].to_list()
        curr_ouput = list(map(lambda x,y,z: (x+ '_' + y, z) , \
                              csv_size*[i_col], col_vals, csv_size*[document_name]))
        output_list += curr_ouput
    
    return output_list

Write a reduce function `inverted_reduce(value, documents)`, where the field “documents” contains a list of all CSV documents per given value.   
This list might have duplicates.   
Reduce function will return new list without duplicates.

For example,  
calling the function `inverted_reduce(‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’)`   
will return a list `[‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’]`

In [18]:
def inverted_reduce(value, documents):
    ducument_name_list = documents.split(',')
    ducument_name_list_no_duplicates = list(set(ducument_name_list))
    string_ducument_name_list_no_duplicates = (', ').join(ducument_name_list_no_duplicates)
    return_list = [value, string_ducument_name_list_no_duplicates]
    return return_list

<br><br><br><br>
# Question 4
## Testing Your MapReduce

**Create Python list** `input_data` : `[‘myCSV1.csv’,.. ,‘myCSV20.csv’]`

In [19]:
input_data = glob.glob(csv_file_name+'*'+csv_ending)
print(f'input_data:\n{input_data}')

input_data:
['myCSV1.csv', 'myCSV0.csv', 'myCSV2.csv', 'myCSV3.csv', 'myCSV7.csv', 'myCSV6.csv', 'myCSV4.csv', 'myCSV5.csv', 'myCSV19.csv', 'myCSV18.csv', 'myCSV10.csv', 'myCSV11.csv', 'myCSV13.csv', 'myCSV12.csv', 'myCSV16.csv', 'myCSV17.csv', 'myCSV15.csv', 'myCSV14.csv', 'myCSV8.csv', 'myCSV9.csv']


**Submit MapReduce as follows:**

In [23]:
mapreduce = MapReduceEngine()
status = mapreduce.execute(input_data, inverted_map, inverted_reduce)
print(status)

2022-01-07 14:52:29,243 [INFO] lithops.config -- Lithops v2.5.8


Exception: Unsupported Python version: 3.10

Make sure that `MapReduce Completed` should be printed and `mapreducefinal` folder should contain the result files.

**Use python to delete all temporary data from mapreducetemp folder and delete SQLite database:**

# remove temporary folders & files

In [70]:
# delete all csv files
for i_file in input_data:
    if os.path.exists(i_file):
        os.remove(i_file)
if os.path.exists(db_file_name):
    os.remove(db_file_name)
for i_folder in map_reduce_folder_names:
    if os.path.exists(i_folder):
        shutil.rmtree(i_folder)

<br><br><br><br>

# Question 5
# Final Thoughts

The phase where `MapReduceEngine` reads all temporary files generated by maps and sort them to provide each reducer a specific key is called the **shuffle step**.

Please explain **clearly** what would be the main problem of MapReduce when processing Big Data, if there is no shuffle step at all, meaning reducers will directly read responses from the mappers.

## Answer:

> The main problem of MapReduce if there is no shuffle step is:

> **the reduce step will not be performed as expected (we could say it failed)**.

> See image below (from the lecture):

<img src="http://1.bp.blogspot.com/-UvgLSDv7Rb4/Tbpn3veAOTI/AAAAAAAAAVk/kdaMzLa50BE/s1600/WordCountFlow.JPG" width="1000">

<br>
We can see in the example diagram above that if a reduce worker will recieve the output of the map worker directly, then the reduce function will return the wrong output.
<br>
for example:
the bottom map worker will output:

 - apple,1
 
 - apple,1
 
 - plum,1
 
the bottom reduce worker will recieve this as input, and might output:

 - apple,2
 
 - plum,1
 
<br>
After the reduce worker finished there is no other reduce step.
<br>
And so, If we will do the same for all map and reduce workers, we will get the following WRONG unreduced and unsummed output:

 - apple,1
 
 - orange,1
 
 - mango,1
 
 - orange,1
 
 - graspes,1
 
 - plum,1
 
 - apple,1
 
 - plum,1
 
 - mango,1
 
 - apple,2
 
 - plum,1
 
<br>
which is obviously not the wanted output.

<br><br><br><br>
Thank You :)